Dans ce script, nous allons tester différents modèles de type convolution Nelle pour examiner leur performance. Nous allons ensuite tester s'ils sont robustes lorsqu'on applique une rotation à l'image d'entrée.

In [ ]:
import retinoto_py as fovea

args = fovea.Params()
args

In [ ]:

args

# testing each network on the validation dataset

In [ ]:
import torch
import torchvision.models as models


VAL_IMAGE_DIR = args.DATAROOT / 'Imagenet_full' / 'train'

# --- 3. Load the Pre-trained ResNet Model ---
# We'll use ResNet50, a powerful and common choice.
# `pretrained=True` downloads the model weights trained on ImageNet.
model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)

# Move the model to the selected device (GPU or CPU)
model = model.to(args.device)

# Set the model to evaluation mode.
model.eval()

In [ ]:
extensions = ['*.jpeg', '*.jpg', '*.png', '*.JPEG', '*.JPG', '*.PNG']

image_files = []
for ext in extensions:
    # rglob returns a generator, so we extend the list with its results
    image_files.extend(VAL_IMAGE_DIR.rglob(ext))

print(f'In folder {VAL_IMAGE_DIR}, I found {len(image_files)} images')

In [ ]:
import torchvision.transforms as transforms
from PIL import Image
import os
import requests
import json # Don't forget to import json

# --- 4. Download and Load the ImageNet Class Index (with caching) ---
LABELS_URL = 'https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json'
LABELS_FILE = args.data_cache / 'imagenet_class_index.json' # Local cache file name

try:
    # Check if we already have the file
    if not os.path.exists(LABELS_FILE):
        print(f"Downloading labels to {LABELS_FILE}...")
        response = requests.get(LABELS_URL)
        response.raise_for_status()
        with open(LABELS_FILE, 'w') as f:
            json.dump(response.json(), f)
    else:
        print(f"Loading labels from local cache {LABELS_FILE}...")
        
    # In both cases, load from the local file
    with open(LABELS_FILE, 'r') as f:
        class_idx = json.load(f)

    # Create a simple mapping from index to class name for easy lookup
    idx2label = [class_idx[str(k)][1] for k in range(len(class_idx))]

except requests.exceptions.RequestException as e:
    print(f"Error downloading labels: {e}")
    exit()
except (IOError, json.JSONDecodeError) as e:
    print(f"Error handling local label file: {e}")
    exit()
idx2label

In [ ]:
# --- 5. Define Image Pre-processing ---
# The images must be pre-processed in the exact same way the model was trained on.
# This includes resizing, cropping, and normalizing.
preprocess = transforms.Compose([
    transforms.Resize(256),                # Resize the shortest side to 256px
    transforms.CenterCrop(224),            # Crop the center 224x224 pixels
    transforms.ToTensor(),                 # Convert the image to a PyTorch Tensor
    transforms.Normalize(                  # Normalize with ImageNet mean and std
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


In [ ]:

print(f"\nStarting inference on images in '{VAL_IMAGE_DIR}'...")
print("-" * 50)

if not image_files:
    print("No image files found in the specified directory.")
else:
    for image_name in image_files:
        image_path = os.path.join(VAL_IMAGE_DIR, image_name)

        try:
            # Load the image
            img = Image.open(image_path).convert('RGB')

            # Pre-process the image and add a batch dimension
            # PyTorch models expect a batch of images, so we unsqueeze a dimension.
            img_t = preprocess(img)
            batch_t = torch.unsqueeze(img_t, 0).to(args.device)

            # Run inference
            # `torch.no_grad()` tells PyTorch not to compute gradients, saving memory and computation
            with torch.no_grad():
                output = model(batch_t)

            # --- Get the Prediction ---
            # The output is a tensor of logits. We find the index of the highest logit.
            # `torch.max` returns the max value and its index. We only need the index.
            _, index = torch.max(output, 1)
            
            # Move the result to CPU and convert it to a Python integer
            predicted_index = index[0].item()
            
            # Get the human-readable label from our mapping
            predicted_label = idx2label[predicted_index]

            # --- Print the result ---
            print(f"Image: {image_name:<30} | Predicted: {predicted_label}")

        except Exception as e:
            print(f"Could not process {image_name}: {e}")

    print("-" * 50)
    print("Inference complete.")

